# [Spinal Cord Toolbox (SCT)](https://spinalcordtoolbox.com/index.html)

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/overview.png" align="center" width="750px">

## Installation Instructions
The instructions below are for MacOS and Linux platforms. \
For more installation instructions, such as installing on Windows or Docker, check [here](https://spinalcordtoolbox.com/user_section/installation.html).

#### Run the following (starting with $) in your terminal
Skip if you already have SCT and FSLeyes installed.

In [ ]:
# First we'll clone the SCT repository from GitHub 
$git -C spinalcordtoolbox pull || git clone https://github.com/spinalcordtoolbox/spinalcordtoolbox.git spinalcordtoolbox

# Change our directory to the newly created directory
$cd spinalcordtoolbox

#start installation
$./install_sct -y

In [ ]:
# Install the following
!${SCT_DIR}/python/envs/venv_sct/bin/pip install matplotlib-inline
!${SCT_DIR}/python/envs/venv_sct/bin/pip install Ipython

Open a new terminal, and run the following to install FSLeyes.\
[FSLeyes](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLeyes) is a GUI image viewer. We'll need this to view our input/output images, and perform labelling when necessary


In [ ]:
# Run the following to install fsleyes
$source spinalcordtoolbox/python/etc/profile.d/conda.sh
$conda create -c conda-forge -p ~/fsleyes_env fsleyes -y

# Next let's make fsleyes available in the terminal without having to activate its environment
$sudo ln -s ~/fsleyes_env/bin/fsleyes /usr/local/bin/fsleyes

Test if SCT and FSLeyes are successfully installed, and working.

In [ ]:
# Run the following in the terminal
$sct_testing

# Run the following to check for fsleyes...
$fsleyes

In [ ]:
# library to open our results .html instead of from the terminal
import webbrowser

# [Spinal Cord Registration to PAM50 template - Tutorial](https://spinalcordtoolbox.com/user_section/tutorials/registration-to-template.html)

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/template-registration/registration-pipeline.png" align="center" width="750px">

## Phase 1: [Vertebral Labeling](https://spinalcordtoolbox.com/user_section/tutorials/registration-to-template/vertebral-labeling.html#vertebral-labeling-for-anatomical-images)

<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/vertebral-labeling/vertebral-labeling.png" align="center" width="350px">        <img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/vertebral-labeling/vertebral-labeling-conventions.png" align="center" width="300px">


### Labeling algorithm (Fully Automated): **sct_label_vertebrae**

**sct_label_vertebrae** does the following:
* Step 1️⃣: **Straightening** - this step involves straightening of the spinal cord to prepare it for the next steps.

* Step 2️⃣: **C2-C3 disc detection** - C2-C3 disc detection is done early, becuase of its easy-to-detect nature.

* Step 3️⃣: **Labeling of neighbouring discs** - this is done using a similarity measure with reference to the PAM50 template. 

* Step 4️⃣: **Un-straightening** - here, both the spinal cord and the labeled segmentation are un-straightened (back to their original state), and the labels are saved.

And it is *Contrast-independent*, *Produces both label types*, and *Robust to missing discs*.


Input arguments:
* -i: Input image 
* -s: Segmented spinal cord corresponding to the input image
* -c: Contrast of the input image
* -qc: Directory for Quality Control reporting. QC reports allow us to evaluate the results slice-by-slice. 

Output files/folders:
* straight_ref.nii.gz: the straightened input image.  
* warp_curve2straight.nii.gz: 4D warping field of the transformation from the original image to the straightened image.
* warp_straight2curve.nii.gz: 4D warping field of the inverse transformation from the straightened image to original image.
* straightening.cache: cache of the strainghtening process
* t2_seg_labeled.nii.gz: the labeled spinal cord
* t2_seg_labeled_discs.nii.gz: single-voxel intervertebral disc labels (without the segmented spinal cord). 

Without data there's no SCT, so let's download one for this tutorial by running the following.

In [ ]:
# If you're on linux run
!wget https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_vertebral-labeling.zip -O data_vertebral-labeling.zip -q

# Otherwise, if you're on MacOS run
# !curl -L https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_vertebral-labeling.zip -o data_vertebral-labeling.zip

!unzip -o data_vertebral-labeling.zip -d vert_labeling && rm data_vertebral-labeling.zip

In [ ]:
%cd 'vert_labeling/single_subject/data'
pwd = %pwd

In [ ]:
!sct_label_vertebrae -i t2/t2.nii.gz -s t2/t2_seg.nii.gz -c t2 -ofolder vlabeling/full -qc ./vlabeling/qc_singleSubj/t2/full 

In [ ]:
url = "file://" + pwd + "/vlabeling/qc_singleSubj/t2/full/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes t2/t2.nii.gz -a 100.0 ./vlabeling/full/t2_seg_labeled.nii.gz -cm subcortical -a 50.0

### Alternative #1 (Semi-automated): Manually labeling the C2-C3 disc (**sct_label_utils**)

Input arguments:
* -i: the input image
* -create-viewer: opens a GUI where you do the labeling
* -msg: GUI window title
* -o: output file name

Output files/folders:
* label_c2c3.nii.gz: image containing the single-voxel label

The output image can be passed via **initlabel** to the fully automated labeling function **sct_label_vertebrae**. Note: all labels in the algorithm are preserved, except the manually labeled one. label_c2c3.nii.gzlabel_c2c3.nii.gz

In [ ]:
!sct_label_utils -i t2/t2.nii.gz -create-viewer 3 -o ./vlabeling/label_c2c3.nii.gz \
                -msg "Click at the posterior tip of C2/C3 inter-vertebral disc"

In [ ]:
!fsleyes t2/t2.nii.gz ./vlabeling/label_c2c3.nii.gz

In [ ]:
!sct_label_vertebrae -i t2/t2.nii.gz -s t2/t2_seg.nii.gz -c t2 -initlabel ./vlabeling/label_c2c3.nii.gz -qc ./vlabeling/qc_singleSubj/t2/c2c3

In [ ]:
url = "file://" + pwd + "/vlabeling/qc_singleSubj/t2/c2c3/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes t2/t2.nii.gz -a 100.0 t2_seg_labeled.nii.gz -cm subcortical -a 50.0

### Alternative #2 (Fully Manual): Manually labeling all labels

This approach is taken, when automatic labeling can't be applied due to absence of discs or having a heavily artifacted data.

You will pass the various discs to label to the **-create-viewer** argument

In [ ]:
# Example: Label C2-C3, C3-C4, and C4-C5 discs

!sct_label_utils -i t2/t2.nii.gz -create-viewer 3,4,5 -o ./vlabeling/labels_disc.nii.gz \
                -msg "Place labels at the posterior tip of each inter-vertebral disc. E.g. Label 3: C2/C3, Label 4: C3/C4, etc."

In [ ]:
!fsleyes t2/t2.nii.gz ./vlabeling/labels_disc.nii.gz 

### Extracting specific labels for registration (**sct_label_utils**)

Input arguments:
* -i: input image containing a spinal cord labeled with vertebral levels
* -vert-body: the top and bottom vertebral levels to create new single-voxel labels for. Choose labels based on your region of interest.
* -o: Output filename

Output files/folders:
* t2_labels_vert.nii.gz: Image containing 2 single-voxel vertebral labels, centered within the chosen region vertebral bodies.

In [ ]:
# Example: Extract C3 to T9 region
!sct_label_utils -i t2_seg_labeled.nii.gz -vert-body 3,9 -o ./vlabeling/t2_labels_vert.nii.gz

In [ ]:
!fsleyes t2_seg_labeled.nii.gz ./vlabeling/t2_labels_vert.nii.gz 

## Phase 2: [Registering to the PAM50 template](https://spinalcordtoolbox.com/user_section/tutorials/registration-to-template/template-registration.html#registering-labeled-anatomical-images-to-the-pam50-template)

Useful links:
* [PAM50 Template](https://spinalcordtoolbox.com/overview/concepts/pam50.html#pam50)     
* [Warping fields](https://spinalcordtoolbox.com/overview/concepts/warping-fields.html#warping-fields)

### Registration algorithm: **sct_register_to_template**
**sct_register_to_template** does the following:
1️⃣ Straightening
2️⃣ Vertebrae-matching transformation
3️⃣ Shape-matching transformation

Input arguments:
* -i: input image
* -s: segmented spinal cord of input image
* -l: One or two labels located at the center of the spinal cord, on the mid-vertebral slice
* -c: input image contrast. This determines which template to use
+ -qc: quality control directory name

Output files/folders:
* anat2template.nii.gz: subject image warped to the template space.
* template2anat.nii.gz : template image warped to the subject space.
* warp_anat2template.nii.gz : 4D warping field that defines the transform from the subject image to the template image.
* warp_template2anat.nii.gz : 4D warping field that defines the inverse transform from the template image to the subject image.


<img src="https://raw.githubusercontent.com/spinalcordtoolbox/doc-figures/master/template-registration/io-sct_register_to_template.png" align="center" width="550px"> 

In [ ]:
%cd ../../../

In [ ]:
# If you're on linux run
!wget https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_template-registration.zip -q

# Otherwise, if you're on MacOS run
# !curl -L https://github.com/spinalcordtoolbox/sct_tutorial_data/releases/download/r20220125/data_template-registration.zip -o data_template-registration.zip

!unzip -o data_template-registration.zip -d PAM5O_regn && rm data_template-registration.zip

In [ ]:
%cd 'PAM5O_regn/single_subject/data'
pwd = %pwd

In [ ]:
!sct_register_to_template -i t2/t2.nii.gz -s t2/t2_seg.nii.gz -l t2/t2_labels_vert.nii.gz -c t2 -qc ./registration/qc_singleSubj/t2

In [ ]:
url = "file://" + pwd + "/registration/qc_singleSubj/t2/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes t2/t2.nii.gz template2anat.nii.gz

In [ ]:
!fsleyes ${SCT_DIR}/data/PAM50/template/PAM50_t2.nii.gz anat2template.nii.gz

### [Transforming the template using warping fields](https://spinalcordtoolbox.com/user_section/tutorials/registration-to-template/template-registration/applying-the-warping-fields.html#transforming-the-template-using-warping-fields): **sct_warp_template**

Input arguments
* -d : Destination image the template will be warped to.
* -w : Warping field (template space to anatomical space).
* -a : Whether or not to also warp the white matter atlas. (If -a 1 is specified, label/atlas/ will also be warped.)
* -qc : Directory for Quality Control reporting. QC reports allow us to evaluate the results slice-by-slice.

Output files/folders
* label/template/ : This directory contains the 15 PAM50 template objects that have been transformed into the subject space 

In [ ]:
!sct_warp_template -d t2/t2.nii.gz -w ./warp_template2anat.nii.gz -a 0 -qc ./transformed/qc_singleSubj

In [ ]:
url = "file://" + pwd + "/transformed/qc_singleSubj/index.html"
webbrowser.open(url)

In [ ]:
!fsleyes t2/t2.nii.gz -cm greyscale -a 100.0 label/template/PAM50_t2.nii.gz -cm greyscale -dr 0 4000 -a 100.0 label/template/PAM50_gm.nii.gz -cm red-yellow -dr 0.4 1 -a 50.0 label/template/PAM50_wm.nii.gz -cm blue-lightblue -dr 0.4 1 -a 50.0

## Congratulations for completing this tutorial 🍾💐